In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

from ultralytics import YOLO

In [3]:
alpha = 0.9
prev_middle_x, prev_middle_y = None, None

In [4]:
model = YOLO("yolov8n.pt")

In [17]:
def roi(target):
    pts = np.array([[420, 1080],[750, 800],[1200, 800],[1550, 1080]], np.int32)    # ROI 영역 지정
    mask = np.zeros_like(target)
    cv2.fillConvexPoly(mask, pts, (255,255,255))
    masked = cv2.bitwise_and(target, mask)
    return masked

def zeros_like(target, contours):
    task = np.zeros_like(target)
    cv2.drawContours(task, contours, -1, (0,255,0), 0)
    return task

In [15]:
def merge_lines(lines, angle_threshold=10):
    merged_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        
        if not merged_lines:
            merged_lines.append((x1, y1, x2, y2, angle))
        else:
            merged = False
            for i, (mx1, my1, mx2, my2, mangle) in enumerate(merged_lines):
                if abs(angle - mangle) < angle_threshold:
                    nx1, ny1 = min(x1, mx1), min(y1, my1)
                    nx2, ny2 = max(x2, mx2), max(y2, my2)
                    merged_lines[i] = (nx1, ny1, nx2, ny2, mangle)
                    merged = True
                    break
            if not merged:
                merged_lines.append((x1, y1, x2, y2, angle))
    return [(int(x1), int(y1), int(x2), int(y2)) for x1, y1, x2, y2, angle in merged_lines]

In [ ]:
video = './src/video.mp4'
cap = cv2.VideoCapture(video)

distance_threshold = 200 

while True:
    ret, t_frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(t_frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 80, 80)

    roi_frame = roi(edges)
    lines = cv2.HoughLinesP(roi_frame, 1, np.pi / 180, threshold=50, minLineLength=100, maxLineGap=20)

    if lines is not None:
        merged_lines = merge_lines(lines)
        line_centers = []

        for x1, y1, x2, y2 in merged_lines:
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            line_centers.append((center_x, center_y))

        if line_centers:
            left_center = min(line_centers, key=lambda point: point[0])
            right_center = max(line_centers, key=lambda point: point[0])

            middle_x = (left_center[0] + right_center[0]) // 2
            middle_y = (left_center[1] + right_center[1]) // 2

            if prev_middle_x is None or prev_middle_y is None:
                prev_middle_x, prev_middle_y = middle_x, middle_y
            else:
                middle_x = int(alpha * prev_middle_x + (1 - alpha) * middle_x)
                middle_y = int(alpha * prev_middle_y + (1 - alpha) * middle_y)
                prev_middle_x, prev_middle_y = middle_x, middle_y

            cv2.circle(t_frame, (middle_x, middle_y), 10, (100, 144, 250), -1)
    
    contours, hierachy = cv2.findContours(roi_frame, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(t_frame, contours, -1, (248,144,0), 0)

    results = model(t_frame)
    warning = False

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            class_id = int(box.cls[0])
            confidence = box.conf[0] * 100

            box_height = y2 - y1

            if box_height > distance_threshold:
                warning = True

            label = f"{model.names[class_id]}: {int(confidence)}%"
            color = (0, 255, 0)
            cv2.rectangle(t_frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(t_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
    if warning:
        cv2.putText(t_frame, "Warning!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

    cv2.imshow("YOLOv8 Detection", t_frame)

    key = cv2.waitKey(13)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 1 car, 48.0ms
Speed: 2.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 56.0ms
Speed: 2.0ms preprocess, 56.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 56.0ms
Speed: 3.0ms preprocess, 56.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 47.5ms
Speed: 2.0ms preprocess, 47.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 47.0ms
Speed: 2.0ms preprocess, 47.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 48.0ms
Speed: 2.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 47.0ms
Speed: 3.0ms preprocess, 47.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 47.0ms
Speed: 2.0ms preprocess, 47.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ca